# Predicting Particulate Matter (PM2.5) Concentrations in the Air of China

# Implementation

In [71]:
# Import libraries necessary for this project
import csv
import numpy as np
import pandas as pd
from IPython.display import display # Allows the use of display() for DataFrames
from time import time
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# Dara preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from math import sqrt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import cross_validate
# Import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import LeaveOneGroupOut

from matplotlib import pyplot as plt
from sklearn import svm
# Pretty display for notebooks
%matplotlib inline
from math import sqrt



In [72]:
# Load Guangzhou dataset
Guangzhou = pd.read_csv('dataset_Guangzhou_clean.csv')
Guangzhou = Guangzhou.drop('Unnamed: 0', axis = 1)

# Load Beijing dataset
Beijing = pd.read_csv('dataset_Beijing_clean.csv')
Beijing = Beijing.drop('Unnamed: 0', axis = 1)


# Load Chengdu dataset
Chengdu = pd.read_csv('dataset_Chengdu_clean.csv')
Chengdu = Chengdu.drop('Unnamed: 0', axis = 1)

# Load Shanghai dataset
Shanghai = pd.read_csv('dataset_Shanghai_clean.csv')
Shanghai = Shanghai.drop('Unnamed: 0', axis = 1)

# Load Shenyang dataset
Shenyang = pd.read_csv('dataset_Shenyang_clean.csv')
Shenyang = Shenyang.drop('Unnamed: 0', axis = 1)

dataset = Beijing.append(Chengdu, ignore_index=True, sort=True)
dataset = dataset.append(Shanghai, ignore_index=True, sort=True)
dataset = dataset.append(Shenyang, ignore_index=True, sort=True)
dataset = dataset.append(Guangzhou, ignore_index=True, sort=True)

Beijing_testing_rows = Beijing.sample(frac=0.2)
Beijing = Beijing.drop(Beijing.index[Beijing_testing_rows.index])

Guangzhou_testing_rows = Guangzhou.sample(frac=0.2)
Guangzhou = Guangzhou.drop(Guangzhou.index[Guangzhou_testing_rows.index])


Chengdu_testing_rows = Chengdu.sample(frac=0.2)
Chengdu = Chengdu.drop(Chengdu.index[Chengdu_testing_rows.index])


Shanghai_testing_rows = Shanghai.sample(frac=0.2)
Shanghai = Shanghai.drop(Shanghai.index[Shanghai_testing_rows.index])


Shenyang_testing_rows = Shenyang.sample(frac=0.2)
Shenyang = Shenyang.drop(Shenyang.index[Shenyang_testing_rows.index])


train_dataset = Beijing.append(Chengdu, ignore_index=True, sort=True)
train_dataset = train_dataset.append(Shanghai, ignore_index=True, sort=True)
train_dataset = train_dataset.append(Shenyang, ignore_index=True, sort=True)
train_dataset = train_dataset.append(Guangzhou, ignore_index=True, sort=True)

test_dataset = Beijing_testing_rows.append(Chengdu_testing_rows, ignore_index=True, sort=True)
test_dataset = test_dataset.append(Shanghai_testing_rows, ignore_index=True, sort=True)
test_dataset = test_dataset.append(Shenyang_testing_rows, ignore_index=True, sort=True)
test_dataset = test_dataset.append(Guangzhou_testing_rows, ignore_index=True, sort=True)


# Total number of records
n_records = len(dataset["PM_US Post"])
print("Number of records for all Chines cities: ", n_records)
print("*********************")

# Total number of records
n_records = len(train_dataset["PM_US Post"])
print("Number of records for all Chines cities: Training : ", n_records)
print("*********************")
display(train_dataset.info())
display(train_dataset.describe())

# Total number of records
n_records = len(test_dataset["PM_US Post"])
print("Number of records for all Chines cities: Testing : ", n_records)
print("*********************")
display(test_dataset.info())
display(test_dataset.describe())
#print(len(Beijing))
#print(len(Beijing_testing_fraction_of_rows))
#print((Beijing_testing_rows['year'].value_counts()))
#print(Beijing.head(n = 5))
#print(Beijing_testing_rows.head(n = 5))

Number of records for all Chines cities:  117200
*********************
Number of records for all Chines cities: Training :  93761
*********************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93761 entries, 0 to 93760
Data columns (total 28 columns):
DEWP             93761 non-null float64
HUMI             93761 non-null float64
PM_US Post       93761 non-null float64
PRES             93761 non-null float64
TEMP             93761 non-null float64
Weekdays         93761 non-null int64
Weekends         93761 non-null int64
cbwd_NE          93761 non-null int64
cbwd_NW          93761 non-null int64
cbwd_SE          93761 non-null int64
cbwd_SW          93761 non-null int64
cbwd_cv          93761 non-null int64
day              93761 non-null int64
day_cos          93761 non-null float64
day_sin          93761 non-null float64
hour             93761 non-null int64
hour_cos         93761 non-null float64
hour_sin         93761 non-null float64
month            93761 non-null int64

None

,DEWP,HUMI,PM_US Post,PRES,TEMP,Weekdays,Weekends,cbwd_NE,cbwd_NW,cbwd_SE,...,month,month_cos,month_sin,new_wind,precipitation,season_1,season_2,season_3,season_4,year
count,93761.000000,93761.000000,93761.000000,93761.000000,93761.000000,93761.000000,93761.000000,93761.000000,93761.000000,93761.000000,...,93761.000000,9.376100e+04,93761.000000,93761.000000,93761.000000,93761.000000,93761.000000,93761.000000,93761.000000,93761.000000
mean,8.973951,66.601146,71.017897,1013.578165,16.175369,0.716033,0.283967,0.222001,0.246979,0.221179,...,6.533740,7.335476e-03,-0.008719,2.757414,0.124664,0.248835,0.242094,0.254530,0.254541,2014.009236
std,12.326445,22.355580,65.313242,9.907536,10.745159,0.450923,0.450923,0.415594,0.431257,0.415043,...,3.476925,7.053905e-01,0.708735,1.944073,1.120329,0.432340,0.428353,0.435599,0.435605,0.809671
min,-40.000000,2.000000,1.000000,975.000000,-25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,-1.000000e+00,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2013.000000
25%,1.000000,51.560000,29.000000,1006.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.000000,-5.000000e-01,-0.866025,1.100000,0.000000,0.000000,0.000000,0.000000,0.000000,2013.000000
50%,12.000000,70.360000,52.000000,1013.000000,18.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,6.000000,-1.836970e-16,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2014.000000
75%,19.000000,86.000000,90.000000,1021.000000,24.400000,1.000000,1.000000,0.000000,0.000000,0.000000,...,10.000000,8.660254e-01,0.500000,4.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2015.000000
max,28.000000,100.000000,932.000000,1046.000000,42.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,12.000000,1.000000e+00,1.000000,20.120000,48.600000,1.000000,1.000000,1.000000,1.000000,2015.000000


Number of records for all Chines cities: Testing :  23439
*********************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23439 entries, 0 to 23438
Data columns (total 28 columns):
DEWP             23439 non-null float64
HUMI             23439 non-null float64
PM_US Post       23439 non-null float64
PRES             23439 non-null float64
TEMP             23439 non-null float64
Weekdays         23439 non-null int64
Weekends         23439 non-null int64
cbwd_NE          23439 non-null int64
cbwd_NW          23439 non-null int64
cbwd_SE          23439 non-null int64
cbwd_SW          23439 non-null int64
cbwd_cv          23439 non-null int64
day              23439 non-null int64
day_cos          23439 non-null float64
day_sin          23439 non-null float64
hour             23439 non-null int64
hour_cos         23439 non-null float64
hour_sin         23439 non-null float64
month            23439 non-null int64
month_cos        23439 non-null float64
month_sin        23439 non-null

None

,DEWP,HUMI,PM_US Post,PRES,TEMP,Weekdays,Weekends,cbwd_NE,cbwd_NW,cbwd_SE,...,month,month_cos,month_sin,new_wind,precipitation,season_1,season_2,season_3,season_4,year
count,23439.000000,23439.000000,23439.000000,23439.000000,23439.000000,23439.00000,23439.00000,23439.000000,23439.000000,23439.000000,...,23439.000000,2.343900e+04,23439.000000,23439.000000,23439.000000,23439.000000,23439.000000,23439.000000,23439.000000,23439.000000
mean,9.049435,66.617170,71.075643,1013.529634,16.257144,0.71108,0.28892,0.219335,0.246768,0.225692,...,6.553863,2.402283e-03,-0.011494,2.757413,0.123580,0.248944,0.244550,0.257989,0.248517,2014.019625
std,12.285323,22.350348,66.554350,9.900487,10.692797,0.45327,0.45327,0.413805,0.431140,0.418046,...,3.463694,7.034655e-01,0.710662,1.940366,1.107444,0.432411,0.429829,0.437537,0.432163,0.810192
min,-38.000000,2.000000,1.000000,980.300000,-25.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,1.000000,-1.000000e+00,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2013.000000
25%,1.600000,51.560000,29.000000,1006.000000,9.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,4.000000,-5.000000e-01,-0.866025,1.100000,0.000000,0.000000,0.000000,0.000000,0.000000,2013.000000
50%,12.000000,70.470000,52.000000,1013.000000,18.000000,1.00000,0.00000,0.000000,0.000000,0.000000,...,6.000000,-1.836970e-16,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2014.000000
75%,19.000000,85.230000,90.000000,1021.000000,24.600000,1.00000,1.00000,0.000000,0.000000,0.000000,...,10.000000,5.000000e-01,0.500000,4.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2015.000000
max,28.000000,100.000000,756.000000,1046.000000,40.000000,1.00000,1.00000,1.000000,1.000000,1.000000,...,12.000000,1.000000e+00,1.000000,15.200000,35.000000,1.000000,1.000000,1.000000,1.000000,2015.000000


In [73]:
dataset.to_csv('dataset_all_cities_clean.csv')

In [57]:
# Machine learning algorithms decleration
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor


LR = LinearRegression()
RF = RandomForestRegressor(n_estimators = 96)
ANN = MLPRegressor(hidden_layer_sizes= (128, 256))
SVR = SVR(kernel='rbf')
GNB = GaussianNB()
KNN = KNeighborsRegressor(n_neighbors=5)
#SVR_tuned = SVR(kernel='rbf', C = 707, epsilon = 4)

MLs = {'LR' : LR, 'RF': RF, 'ANN' : ANN, 'SVR' : SVR}
#MLs = {'LR' : LR}


In [58]:


def apply_L5(cityTrainName, Train_dataset, cityTestName, Test_dataset, MLname, estimator, f_out, Un_needed_columns):
    # Construct the pipeline with a standard scaler and a small neural network
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append((MLname, estimator))
    model = Pipeline(estimators)

    
    # Split the data into features and target label
    #Un_needed_columns = ['PM_US Post']
    
    # Split the data into features and target label
    X_train = Train_dataset.drop(Un_needed_columns, axis = 1)
    X_test = Test_dataset.drop(Un_needed_columns, axis = 1)
    
    y_train = Train_dataset['PM_US Post']
    y_test = Test_dataset['PM_US Post']
    
    # Saving feature names for later use
    features_list = list(X_train.columns)

    print(estimators)

    print("**Train Split **")
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    R2 = r2_score(y_test, predict)
    MSE =  mean_squared_error(y_test,predict)
    MAE =  mean_absolute_error(y_test,predict)
    RMSE = sqrt(MSE)


    print("Train City : ", cityTrainName)
    print("Test City : ", cityTestName)
    print("MSE : ", MSE)
    print("MAE : ", MAE)
    print("R2 : ", R2)
    print("RMSE : ", RMSE)

    print("-----------------------------------------------")

    
    f_out.write(str(cityTrainName) + ",")
    f_out.write(str(cityTestName) + ",")
    f_out.write(str(MLname) + ",")
    f_out.write('TTS' + ",")
    f_out.write(str(abs(MAE)) + ",")
    f_out.write(str(abs(MSE)) + ",")
    f_out.write(str(RMSE) + ",")
    f_out.write(str(R2) + ",")
    f_out.write(str(len(features_list)) + ",")
    for feature in features_list:
        f_out.write(feature + "&")
    f_out.write("\n")
    

    return features_list

In [59]:


def applyCV(cityTrainName, dataset, cityTestName, MLname, estimator, f_out, Un_needed_columns):
    # Construct the pipeline with a standard scaler and a small neural network
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append((MLname, estimator))
    model = Pipeline(estimators)

    # Split the data into features and target label
    #Un_needed_columns = ['PM_US Post', 'day']
    
    # Split the data into features and target label
    X = dataset.drop(Un_needed_columns, axis = 1)
    y = dataset['PM_US Post']
    
    # Saving feature names for later use
    features_list = list(X.columns)

    # We'll use 5-fold cross validation. That is, a random 80% of the data will be used
    # to train the model, and the prediction score will be computed on the remaining 20%.
    # This process is repeated five times such that the training sets in each "fold"
    # are mutually orthogonal.
    
    K = 3
    kfold = KFold(n_splits=K,  shuffle=True)

    print(estimators)

    print("**cross_val_score + KFold **")

    results_R2 = cross_val_score(model, X, y, cv=kfold, scoring='r2')
    R2 = np.mean(results_R2)
    print('CV Scoring Result: r2 : mean=',np.mean(results_R2),'std=',np.std(results_R2))
    #print(results_R2) 
    print("**************")
    
    results_MAE = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_absolute_error')
    MAE = np.mean(results_MAE)
    print('CV Scoring Result: MAE : mean=',np.mean(results_MAE),'std=',np.std(results_MAE))
    #print(results_MAE)  
    print("**************")

    #results_MSE = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')
    MSE = 0
    #MSE = np.mean(results_MSE)
    #print('CV Scoring Result: MSE : mean=',np.mean(results_MSE),'std=',np.std(results_MSE))
    #print(results_MSE) 
    RMSE = 0
    #RMSE = sqrt(abs(MSE))

    print("-----------------------------------------------")

    
    f_out.write(str(cityTrainName) + ",")
    f_out.write(str(cityTestName) + ",")
    f_out.write(str(MLname) + ",")
    f_out.write('CV(' + str(K) + "),")
    f_out.write(str(abs(MAE)) + ",")
    f_out.write(str(abs(MSE)) + ",")
    f_out.write(str(RMSE) + ",")
    f_out.write(str(R2) + ",")
    f_out.write(str(len(features_list)) + ",")
    for feature in features_list:
        f_out.write(feature + "&")
    f_out.write("\n")


In [ ]:


def applyLOO(cityTrainName, dataset, cityTestName, MLname, estimator, f_out, Un_needed_columns):
    # Construct the pipeline with a standard scaler and a small neural network
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append((MLname, estimator))
    model = Pipeline(estimators)

    # Split the data into features and target label
    #Un_needed_columns = ['PM_US Post', 'day']
    
    # Split the data into features and target label
    X = dataset.drop(Un_needed_columns, axis = 1)
    y = dataset['PM_US Post']
    
    # Saving feature names for later use
    features_list = list(X.columns)

    # We'll use 5-fold cross validation. That is, a random 80% of the data will be used
    # to train the model, and the prediction score will be computed on the remaining 20%.
    # This process is repeated five times such that the training sets in each "fold"
    # are mutually orthogonal.
    
    groups = dataset['Id']
    cv_gen=LeaveOneGroupOut().split(X, y, groups)
    cv = list(cv_gen)

    print(estimators)

    print("**cross_val_score + KFold **")

    results_R2 = cross_val_score(model, X, y, cv=cv, scoring='r2')
    R2 = np.mean(results_R2)
    print('CV Scoring Result: r2 : mean=',np.mean(results_R2),'std=',np.std(results_R2))
    #print(results_R2) 
    print("**************")
    
    results_MAE = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error')
    MAE = np.mean(results_MAE)
    print('CV Scoring Result: MAE : mean=',np.mean(results_MAE),'std=',np.std(results_MAE))
    #print(results_MAE)  
    print("**************")

    #results_MSE = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')
    MSE = 0
    #MSE = np.mean(results_MSE)
    #print('CV Scoring Result: MSE : mean=',np.mean(results_MSE),'std=',np.std(results_MSE))
    #print(results_MSE) 
    RMSE = 0
    #RMSE = sqrt(abs(MSE))

    print("-----------------------------------------------")

    
    f_out.write(str(cityTrainName) + ",")
    f_out.write(str(cityTestName) + ",")
    f_out.write(str(MLname) + ",")
    f_out.write('CV(' + str(K) + "),")
    f_out.write(str(abs(MAE)) + ",")
    f_out.write(str(abs(MSE)) + ",")
    f_out.write(str(RMSE) + ",")
    f_out.write(str(R2) + ",")
    f_out.write(str(len(features_list)) + ",")
    for feature in features_list:
        f_out.write(feature + "&")
    f_out.write("\n")


In [65]:
with open("../China/China_Results/Paper_Results/Level_5_Balanced_Test_22_dayType_features.csv", 'w') as f_out:
    out_colnames = ['Train Site', 'Test Site', 'Algorithm', 'CV', 'MAE', 'MSE', 'RMSE', 'R^2', 'Features_Count', 'Features']        
    writer = csv.DictWriter(f_out, fieldnames = out_colnames)
    writer.writeheader()
    
    Train_city_Name = "Beijing & Chengdu & Shanghai & Shenyang & Guangzhou"
    Test_city_Name = "Beijing & Chengdu & Shanghai & Shenyang & Guangzhou"
    
    for MLname, ML in MLs.items():
        print(Train_city_Name, " ********************** and *********************  ", Test_city_Name)
        Un_needed_columns = ['PM_US Post','day_cos', 'day_sin']
        apply_L5(Train_city_Name, train_dataset, Test_city_Name, test_dataset, MLname, ML, f_out, Un_needed_columns)
        applyCV(Train_city_Name, dataset, Test_city_Name, MLname, ML, f_out, Un_needed_columns)

    


Beijing & Chengdu & Shanghai & Shenyang & Guangzhou  ********************** and *********************   Beijing & Chengdu & Shanghai & Shenyang & Guangzhou
[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LR', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))]
**Train Split **
Train City :  Beijing & Chengdu & Shanghai & Shenyang & Guangzhou
Test City :  Beijing & Chengdu & Shanghai & Shenyang & Guangzhou
MSE :  3198.293315615729
MAE :  38.62402029630035
R2 :  0.24554333590072586
RMSE :  56.55345538175125
-----------------------------------------------
[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LR', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))]
**cross_val_score + KFold **


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to floa

CV Scoring Result: r2 : mean= 0.24087153417010254 std= 0.0006964944532678831
**************


/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to f

CV Scoring Result: MAE : mean= -38.84667873169896 std= 0.04939704730660061
**************
-----------------------------------------------
Beijing & Chengdu & Shanghai & Shenyang & Guangzhou  ********************** and *********************   Beijing & Chengdu & Shanghai & Shenyang & Guangzhou
[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('RF', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=96, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]
**Train Split **


...

Train City :  Beijing & Chengdu & Shanghai & Shenyang & Guangzhou
Test City :  Beijing & Chengdu & Shanghai & Shenyang & Guangzhou
MSE :  2983.6912015267544
MAE :  32.730169385742315
R2 :  0.29616658371658466
RMSE :  54.623174583017
-----------------------------------------------
[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('SVR', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False))]
**cross_val_score + KFold **


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to floa

CV Scoring Result: r2 : mean= 0.28325825217188894 std= 0.0010290885154593439
**************


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to floa

CV Scoring Result: MAE : mean= -33.22634702825169 std= 0.10471634026033942
**************
-----------------------------------------------


In [8]:
print(best_estimator)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)


# Feature Selection 

# Wrapper

In [35]:


def apply_Feature_Selection(cityTrainName, Train_dataset, cityTestName, Test_dataset, MLname, estimator, f_out):
    # Construct the pipeline with a standard scaler and a small neural network
    
    #estimators = []
    #estimators.append(('standardize', StandardScaler()))
    #estimators.append((MLname, estimator))
    #model = Pipeline(estimators)
    model = estimator
    
    # Split the data into features and target label
    Un_needed_columns = ['PM_US Post', 'day']
    
    # Split the data into features and target label
    X_train = Train_dataset.drop(Un_needed_columns, axis = 1)
    X_test = Test_dataset.drop(Un_needed_columns, axis = 1)
    
    y_train = Train_dataset['PM_US Post']
    y_test = Test_dataset['PM_US Post']
    
    # Saving feature names for later use
    feature_list = list(X_train.columns)
    
    # Feature Scaling
    Sc_X = StandardScaler()
    X_train = Sc_X.fit_transform(X_train)
    X_test = Sc_X.transform(X_test)
    
    
    print("**Train Split **")
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    R2 = r2_score(y_test, predict)
    MSE =  mean_squared_error(y_test,predict)
    MAE =  mean_absolute_error(y_test,predict)

    print("Train City : ", cityTrainName)
    print("Test City : ", cityTestName)
    print("MSE : ", MSE)
    print("MAE : ", MAE)
    print("R2 : ", R2)

    
    
    
    if(MLname == 'SVR'):
        

        def f_importances(coef, names):
            imp = coef
            imp,names = zip(*sorted(zip(imp,names)))
            plt.barh(range(len(names)), imp, align='center')
            plt.yticks(range(len(names)), names)
            plt.show()

        f_importances(model.coef_, feature_list)
    
    
    if(MLname == 'RF'):
        # Get numerical feature importances
        importances = list(model.feature_importances_)

        # List of tuples with variable and importance
        feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

        # Sort the feature importances by most important first
        feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

        # Print out the feature and importances 
        [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]
    
    print("-----------------------------------------------")

    f_out.write(str(cityTrainName) + ",")
    f_out.write(str(cityTestName) + ",")
    f_out.write(str(MLname) + ",")
    f_out.write(str(MAE) + ",")
    f_out.write(str(MSE) + ",")
    f_out.write(str(R2) + ",")
    f_out.write(str(('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances) + "\n")

    
    return feature_list

In [36]:
with open("../China/China_Results/After_fixing_outliers/Feature_Selection/Level_5_Balanced_Test_FS_Wrapper.csv", 'w') as f_out:
    out_colnames = ['Train Site', 'Test Site', 'Algorithm', 'MAE', 'MSE', 'R^2', 'Features & importance']        
    writer = csv.DictWriter(f_out, fieldnames = out_colnames)
    writer.writeheader()
    
    Train_city_Name = "Beijing & Chengdu & Shanghai & Shenyang & Guangzhou"
    Test_city_Name = "Beijing & Chengdu & Shanghai & Shenyang & Guangzhou"
    
    MLs = {'RF': RF, 'SVR' : SVR}

    for MLname, ML in MLs.items():
        print(Train_city_Name, " ********************** and *********************  ", Test_city_Name)
        features_list = apply_Feature_Selection(Train_city_Name, train_dataset, Test_city_Name, test_dataset, MLname, ML, f_out)
    
    for feature in features_list:
        f_out.write(feature + "&")


Beijing & Chengdu & Shanghai & Shenyang & Guangzhou  ********************** and *********************   Beijing & Chengdu & Shanghai & Shenyang & Guangzhou
**Train Split **
Train City :  Beijing & Chengdu & Shanghai & Shenyang & Guangzhou
Test City :  Beijing & Chengdu & Shanghai & Shenyang & Guangzhou
MSE :  1412.9684605444725
MAE :  23.49263407247756
R2 :  0.6709652137459938
Variable: new_wind             Importance: 0.18
Variable: TEMP                 Importance: 0.17
Variable: HUMI                 Importance: 0.12
Variable: PRES                 Importance: 0.12
Variable: DEWP                 Importance: 0.1
Variable: hour_cos             Importance: 0.05
Variable: hour_sin             Importance: 0.05
Variable: month_cos            Importance: 0.05
Variable: month_sin            Importance: 0.04
Variable: year                 Importance: 0.04
Variable: cbwd_NE              Importance: 0.01
Variable: cbwd_NW              Importance: 0.01
Variable: cbwd_SE              Importance: 0.

AttributeError: coef_ is only available when using a linear kernel

# Lasso 

In [44]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
  
# Split the data into features and target label
Features = dataset.drop('PM_US Post', axis = 1)
Target = dataset['PM_US Post']

feature_name = Features.columns.tolist()

display(Features.head(n=2))
display(Target.head(n=2))

scaler = StandardScaler()
ScaledFeatures = scaler.fit_transform(Features)
  
lasso = Lasso(alpha=.3)
lasso.fit(Features, Target)
  
print ("Lasso model: ", pretty_print_linear(lasso.coef_, feature_name, sort = True))

,DEWP,HUMI,PRES,TEMP,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_SW,cbwd_cv,day,...,hour_sin,month_cos,month_sin,new_wind,precipitation,season_1,season_2,season_3,season_4,year
0,-10.0,67.0,1018.0,-5.0,0,1,0,0,0,1,...,0.000000,1.0,0.0,4.02,0.0,0,0,0,1,2013
1,-11.0,73.0,1017.0,-7.0,0,1,0,0,0,1,...,0.258819,1.0,0.0,4.02,0.0,0,0,0,1,2013


0    31.0
1    32.0
Name: PM_US Post, dtype: float64

Lasso model:  14.806 * month_cos + -12.392 * cbwd_NE + 11.012 * cbwd_cv + -7.454 * year + -7.159 * new_wind + 6.708 * cbwd_SE + -6.366 * hour_cos + -6.265 * hour_sin + -5.692 * cbwd_NW + 2.402 * season_4 + -2.334 * TEMP + -2.143 * precipitation + 1.294 * month_sin + -1.076 * season_3 + 0.932 * DEWP + 0.375 * day + 0.069 * HUMI + -0.0 * PRES + -0.0 * cbwd_SW + 0.0 * season_1 + -0.0 * season_2


In [45]:
#A helper method for pretty-printing linear models
def pretty_print_linear(coefs, names = None, sort = False):
    if names == None:
        names = ["X%s" % x for x in range(len(coefs))]
    lst = zip(coefs, names)
    if sort:
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))
    return " + ".join("%s * %s" % (round(coef, 3), name)
                                   for coef, name in lst)
 
